In [5]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
# model, layer define

class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [7]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)
# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train:0' shape=() dtype=float32, numpy=5.0>)


In [8]:
# 밀집 linear layer

class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [9]:
# 2개의 layer instance를 만들고 적용하는 전체모델

class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = SequentialModule(name="the_model")

print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


tf.Module 인스턴스는 tf.Variable 또는 할당된 tf.Module 인스턴스를 재귀적으로 자동 수집한다. 이를 토대로 단일 모델 인스턴스로 tf.Module 모음을 관리하고 전체 모델을 저장, 로드할 수 있다. 

In [10]:
print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x0000021A6DF7B460>, <__main__.Dense object at 0x0000021A26581A30>)


In [11]:
for var in my_model.variables:
  print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.16396527,  0.47628602,  0.29108617],
       [-1.3568751 ,  1.1525935 , -2.045172  ],
       [ 0.13822733, -0.51621234, -0.94238317]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.4721115 , -0.7477764 ],
       [-0.14115524, -1.7136995 ],
       [ 1.6012672 ,  1.5600077 ]], dtype=float32)> 



## 변수 생성 연기

특정 입력 형상으로 모듈이 처음 호출될 때까지 변수 생성을 연기하면 입력 크기를 미리 지정할 필요가 없다.

In [12]:
class FlexibleDenseModule(tf.Module):
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.is_built = False
    self.out_features = out_features

  def __call__(self, x):
    if not self.is_built:
      self.w = tf.Variable(
        tf.random.normal([x.shape[-1], self.out_features]), name='w')
      self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
      self.is_built = True

    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [13]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = FlexibleDenseModule(out_features=3)
    self.dense_2 = FlexibleDenseModule(out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.         0.37247378]], shape=(1, 2), dtype=float32)


## 가중치 저장

In [14]:
chkp_path = "weight/my_checkpoint"
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)
checkpoint.write(chkp_path)

'weight/my_checkpoint'

In [15]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

In [17]:
# load
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore(chkp_path)

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.        , 0.37247378]], dtype=float32)>

## 함수 저장

In [18]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  @tf.function
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_model = MySequentialModule(name="the_model")

In [20]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0.       4.301402]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0.       4.301402]
  [0.       4.301402]]], shape=(1, 2, 2), dtype=float32)


In [21]:
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

new_model = MySequentialModule()

tf.summary.trace_on(graph=True, profiler=True)
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
tf.Tensor([[3.178033  1.8347375]], shape=(1, 2), dtype=float32)
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


In [22]:
%tensorboard --logdir logs/func 
# localhost 6006

### SaveModel Creation

완전히 훈련된 모델을 공유하는 방법으론 SaveModel이 권장된다. 함수 모음과 가중치 모음이 모두 포함된다. 

In [25]:
tf.saved_model.save(my_model, "the_saved_model")

INFO:tensorflow:Assets written to: the_saved_model\assets


In [26]:
!dir the_saved_model

 D ����̺��� ����: ���� ��ũ
 ���� �Ϸ� ��ȣ: F808-C512

 d:\Programing\tf\guide\the_saved_model ���͸�

2021-07-25  ���� 11:34    <DIR>          .
2021-07-25  ���� 11:34    <DIR>          ..
2021-07-25  ���� 11:34    <DIR>          assets
2021-07-25  ���� 11:34            14,343 saved_model.pb
2021-07-25  ���� 11:34    <DIR>          variables
               1�� ����              14,343 ����Ʈ
               4�� ���͸�  308,512,628,736 ����Ʈ ����


saved_model.pb파일은 함수형 tf.Graph를 설명하는 프로토콜 버퍼이다. 

In [28]:
# 모델을 새 객체로 load할 수 있다. 
new_model = tf.saved_model.load('the_saved_model')

In [29]:
isinstance(new_model, SequentialModule)

False

저장된 모델을 로드한 new_model은 내부 tf 사용자 객체이다. SequentialModule유형이 아니다. 

In [30]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0.       4.301402]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0.       4.301402]
  [0.       4.301402]]], shape=(1, 2, 2), dtype=float32)


## keras model, layer

### Keras layer

In [31]:
class MyDense(tf.keras.layers.Layer):
  def __init__(self, in_features, out_features, **kwargs):
    super().__init__(**kwargs)

    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

In [32]:
simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[6.7193317, 0.       , 2.4326499]], dtype=float32)>

### Build

In [33]:
class FlexibleDense(tf.keras.layers.Layer):
  def __init__(self, out_features, **kwargs):
    super().__init__(**kwargs)
    self.out_features = out_features

  def build(self, input_shape):  
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

  def call(self, inputs):  
    return tf.matmul(inputs, self.w) + self.b

flexible_dense = FlexibleDense(out_features=3)

In [34]:
flexible_dense.variables

[]

In [35]:
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[-3.211515   3.2916994  4.619859 ]
 [-4.817272   4.937549   6.9297886]], shape=(2, 3), dtype=float32)


In [36]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 0.9111217 ,  1.902284  , -0.14298443],
        [ 0.7788524 , -0.0596848 ,  2.0059624 ],
        [-3.2957315 , -0.19674954,  0.4469516 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

## Keras Model

tf.keras.Model: 완전한 기능을 갖춘 모델 클래스도 제공한다. tf.keras.layers.Layer에서 상속되므로 Keras layer와 같은 방식으로 사용 가능하다. 

In [37]:
class MySequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)

    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)
  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

my_sequential_model = MySequentialModel(name="the_model")

print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[-2.5955515 10.373535 ]], shape=(1, 2), dtype=float32)


In [38]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-1.1804935 , -0.48820588,  1.0369724 ],
        [-0.49254116,  0.44295865, -1.1024046 ],
        [-0.67076445, -0.98139036, -1.0987513 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 1.5447494 , -2.1482692 ],
        [-0.58927387,  1.6819847 ],
        [-1.4755714 , -1.6135299 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [39]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x21a6e5726a0>,
 <__main__.FlexibleDense at 0x21a6cd554f0>)

In [40]:
# 함수형 API가 있는 같은 모델

inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
flexible_dense_3 (FlexibleDe (None, 3)                 12        
_________________________________________________________________
flexible_dense_4 (FlexibleDe (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [41]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[2.9293737, 3.9202614]], dtype=float32)>

## Keras Model 저장

In [42]:
# save
my_sequential_model.save("exname_of_file")

INFO:tensorflow:Assets written to: exname_of_file\assets


In [43]:
# load
reconstructed_model = tf.keras.models.load_model("exname_of_file")

In [44]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.5955515, 10.373535 ]], dtype=float32)>